In [1]:
import csv
import json
import requests
import re
from pathlib import Path

In [2]:
# define model, ollama API, input files
model_name = "deepseek-r1:70b" # model name here
ollama_url = "http://localhost:11434/api/generate"

input_files = [
    "relevance_210725_prompts_templ-1.csv",
    "relevance_210725_prompts_templ-2.csv",
    "relevance_210725_prompts_templ-3.csv",
    "relevance_210725_prompts_templ-4.csv",
    "relevance_210725_prompts_templ-5.csv",
]

In [6]:
import time
import json
import requests
import re
from pathlib import Path
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# How many concurrent requests
MAX_WORKERS = 10

# Helper: call Ollama for a single prompt
def call_ollama_single(prompt: str) -> str:
    payload = {
        "model": model_name,
        "prompt": prompt,
        "stream": False
    }
    resp = requests.post(ollama_url,
                         headers={"Content-Type": "application/json"},
                         data=json.dumps(payload), timeout=60)
    if resp.status_code != 200:
        return f"Error: HTTP {resp.status_code}"
    try:
        result = resp.json()
    except ValueError:
        text = resp.text.strip()
    else:
        # extract text from common fields
        if isinstance(result, dict):
            if "response" in result:
                text = result["response"]
            elif "output" in result:
                text = result["output"]
            elif "responses" in result and isinstance(result["responses"], list):
                # batch-style response
                return result["responses"][0].strip()
            else:
                # chat-style or fallback
                choices = result.get("choices")
                if isinstance(choices, list) and choices:
                    text = choices[0].get("message", {}).get("content", "")
                else:
                    text = str(result)
        elif isinstance(result, list):
            # pure list batch
            return result[0].strip()
        else:
            text = str(result)
    text = str(text).strip()
    # normalize yes/no
    low = text.lower()
    if low.startswith("yes"): return "Yes"
    if low.startswith("no"):  return "No"
    return text

# Process each file
for input_path in input_files:
    # detect template tag and output name
    match = re.search(r"templ-\d+", input_path)
    if not match:
        print(f"Skipping {input_path}, no template tag found.")
        continue
    template_tag = match.group()
    model_tag = model_name.replace(':', '-')
    output_path = f"relevance_210725_completions_{model_tag}-{template_tag}.csv"

    print(f"\nProcessing {input_path} -> {output_path}")
    df = pd.read_csv(input_path, encoding='utf-8')
    # ensure columns exist
    if 'eval_prompt' not in df.columns:
        # assume last column is prompt
        df.rename(columns={df.columns[-1]: 'eval_prompt'}, inplace=True)
    df['eval_completion'] = None
    df['model'] = None

    total = len(df)
    start_all = time.perf_counter()
    completed = 0
    timings = []

    # parallel calls
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = {}
        for idx, prompt in df['eval_prompt'].items():
            futures[executor.submit(call_ollama_single, prompt)] = idx

        for fut in as_completed(futures):
            idx = futures[fut]
            t0 = time.perf_counter()
            try:
                completion = fut.result()
            except Exception as e:
                completion = f"Error: {e}"
            dt = time.perf_counter() - t0
            df.at[idx, 'eval_completion'] = completion
            df.at[idx, 'model'] = model_name

            completed += 1
            timings.append(time.perf_counter() - start_all)
            avg_t = sum(timings) / len(timings)
            remain = total - completed
            eta_sec = remain * avg_t / completed if completed > 0 else 0
            print(f"{completed}/{total} rows done, last={dt:.2f}s, ETA ~{eta_sec/60:.1f}min")

    total_time = time.perf_counter() - start_all
    print(f"Finished {total} rows in {total_time/60:.2f} minutes.")

    # write output
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"Wrote results to {output_path}")



Processing relevance_210725_prompts_templ-1.csv -> relevance_210725_completions_deepseek-r1-70b-templ-1.csv
1/1000 rows done, last=0.00s, ETA ~0.0min
2/1000 rows done, last=0.00s, ETA ~0.0min
3/1000 rows done, last=0.00s, ETA ~0.0min
4/1000 rows done, last=0.00s, ETA ~0.0min
5/1000 rows done, last=0.00s, ETA ~0.0min
6/1000 rows done, last=0.00s, ETA ~0.0min
7/1000 rows done, last=0.00s, ETA ~0.0min
8/1000 rows done, last=0.00s, ETA ~0.0min
9/1000 rows done, last=0.00s, ETA ~0.0min
10/1000 rows done, last=0.00s, ETA ~0.0min
11/1000 rows done, last=0.00s, ETA ~0.0min
12/1000 rows done, last=0.00s, ETA ~0.0min
13/1000 rows done, last=0.00s, ETA ~0.0min
14/1000 rows done, last=0.00s, ETA ~0.0min
15/1000 rows done, last=0.00s, ETA ~0.0min
16/1000 rows done, last=0.00s, ETA ~0.0min
17/1000 rows done, last=0.00s, ETA ~0.0min
18/1000 rows done, last=0.00s, ETA ~0.0min
19/1000 rows done, last=0.00s, ETA ~0.0min
20/1000 rows done, last=0.00s, ETA ~0.0min
21/1000 rows done, last=0.00s, ETA ~0.0m

KeyboardInterrupt: 

In [23]:
# original, took to long

for input_path in input_files:
    # Determine output file name based on template number and model
    template_match = re.search(r"templ-\d+", input_path)
    if not template_match:
        continue
    template_tag = template_match.group() 
    # replace colon with hyphen in models
    model_tag = model_name.replace(":", "-")
    output_path = f"relevance_210725_completions_{model_tag}-{template_tag}.csv"
    
    with open(input_path, newline='', encoding='utf-8') as infile, \
         open(output_path, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        # Read the header and append new column names
        header = next(reader)
        new_header = header + ["eval_completion", "model"]
        writer.writerow(new_header)
        
        # Iterate over each row in the input CSV
        for row in reader:
            if not row:  # skip empty lines if any
                continue
            prompt = row[-1]  # eval_prompt is the last col
            
            # Prepare the JSON payload for Ollama API
            payload = {
                "model": model_name,
                "prompt": prompt,
                "stream": False  # get a single JSON response instead of stream
            }
            
            try:
                response = requests.post(ollama_url, headers={"Content-Type": "application/json"},
                                         data=json.dumps(payload))
            except Exception as e:
                # If there's a connection error or similar, you might want to handle it
                print(f"Error calling Ollama API for prompt: {prompt[:30]}... \n{e}")
                continue
            
            eval_completion = ""
            if response.status_code == 200:
                # Parse JSON response from Ollama
                try:
                    result = response.json()
                except ValueError:
                    # If response is not a valid JSON (unexpected), use raw text
                    result_text = response.text.strip()
                    # Determine yes/no from text
                    if result_text.lower().startswith("yes"):
                        eval_completion = "Yes"
                    elif result_text.lower().startswith("no"):
                        eval_completion = "No"
                    else:
                        eval_completion = result_text  # fallback to whatever it is
                else:
                    # Ollama's response JSON might have the output text in a field.
                    # We attempt common possible keys.
                    if "response" in result:
                        result_text = result["response"]
                    elif "output" in result:
                        result_text = result["output"]
                    elif "content" in result:
                        # If using chat-style response, it might be nested:
                        # e.g., {"model": ..., "choices": [{"message": {"role": "assistant", "content": "Yes"}}], ...}
                        result_text = result.get("content", "") or result.get("message", {}).get("content", "")
                    else:
                        # If none of the known keys, use the full JSON string as fallback
                        result_text = str(result)
                    result_text = str(result_text).strip()
                    # Normalize to "Yes" or "No"
                    if result_text.lower().startswith("yes"):
                        eval_completion = "Yes"
                    elif result_text.lower().startswith("no"):
                        eval_completion = "No"
                    else:
                        eval_completion = result_text
            else:
                # If the API call failed (non-200 status), record the status or an error
                eval_completion = f"Error: HTTP {response.status_code}"
            
            # Append the new columns to the row
            row_with_output = row + [eval_completion, model_name]
            writer.writerow(row_with_output)

    print(f"Completed {input_path} -> {output_path}")

KeyboardInterrupt: 